##### 라이브러리

##### 라이브러리 설치

In [ ]:
%pip install pandas

##### 라이브러리 불러오기

In [ ]:
import pandas as pd
import ast

# 1차 데이터 분석 진행 - 데이터 정리 및 통계, 시각화, 워드 클라우드

## 기사 데이터 정리 및 통계

### 1. 전체 기사 일자별 카운트

##### 삼성전자 뉴스 csv 불러오기

In [ ]:
df_news = pd.read_csv("naver_news/naver_news_20230801_20230831.csv")
df_news.head()

##### date 열 데이터 변경

In [ ]:
df_news["date"] = df_news["date"].apply(lambda x: x.split()[0].replace("-", ""))
df_news.head()

##### 전체 기사 일자별 카운트

In [ ]:
all = []
for day in range(1, 32):
    date_str = f"202308{day:02d}"
    count_all = len(df_news.loc[df_news["date"] == date_str])
    all.append({"날짜":date_str, "전체":count_all})

all

##### 데이터프레임 생성

In [ ]:
count_all = pd.DataFrame(all)
count_all.head()

### 2. 본문이 비어있는 기사 등을 제외하고 크롤링한 기사 중 본문에 삼성전자 글자가 들어간 뉴스 분류

##### content 열에 NaN 값 제거

In [ ]:
df_news.dropna(subset=["content"], inplace=True)
df_news.head()

##### 'content' 열의 데이터에 '삼성전자'가 포함된 행을 필터링하여 날짜별 개수 카운트

In [ ]:
df_samsung = df_news[df_news["content"].str.contains("삼성전자")].groupby("date").size().reset_index(name="본문")
df_samsung.head()

### 3. 삼성 주가데이터 합체 및 주가데이터가 없는 날짜 정리

##### 삼성전자 주식 csv 불러오기

In [ ]:
df_stock = pd.read_csv("naver_stock/naver_stock_20230801_20230831.csv")
df_stock.head()

##### date 열 데이터 변경

In [ ]:
df_stock["date"] = df_stock["date"].apply(lambda x: x.split()[0].replace("-", ""))
df_stock.head()

##### end_price 열 데이터 형식 변경

In [ ]:
df_stock["end_price"] = df_stock["end_price"].str.replace(",", "").astype(float)
df_stock.head()

##### df_stock에서 date와 end_price 열만 선택

In [ ]:
df_stock_subset = df_stock[["date", "end_price"]]
df_stock_subset.head()

##### df_stock의 날짜를 기준으로 데이터프레임 병합

In [ ]:
samsung = pd.merge(df_stock_subset, df_samsung, on="date", how="left")
samsung.head()

##### 열 이름 변경

In [ ]:
samsung = samsung.rename(columns={"date":"날짜", "end_price":"주가"})
samsung.head()

##### 데이터프레임 열 순서 변경

In [ ]:
samsung = samsung[["날짜", "본문", "주가"]]
samsung

##### 인덱스 재설정

In [ ]:
samsung.set_index("날짜", inplace=True)
samsung

### 4. 수치를 백분위로 변경 ((날짜별 합 / 전체 합) * 100)

##### 본문 백분위

In [ ]:
samsung["본문 백분위"] = (samsung["본문"] / samsung["본문"].sum()) * 100
samsung

##### 삼성전자 주가 백분위

In [ ]:
samsung["삼성전자 주가 백분위"] = (samsung["주가"] / samsung["주가"].sum()) * 100
samsung

## 관심도, 카테고리 데이터 통계

### 1. 삼성전자 기사 날짜별 관심도 정리

##### 본문에 삼성전자가 들어간 기사 추출

In [ ]:
df_samsung = df_news[df_news["content"].str.contains("삼성전자")]
df_samsung.head()

##### 인덱스 재설정

In [ ]:
df_samsung.set_index("date", inplace=True)
df_samsung.head()

##### 날짜별 기사 반응 합계 추출

In [ ]:
reactions = {}

for date, dict in df_samsung["reaction"].items():
    reactions_dict = ast.literal_eval(dict)     # 문자열을 딕셔너리로 변환
    reaction_count = 0
    
    # 각 반응에 대해 숫자로 변환하여 더하기
    for count in reactions_dict.values():
        if isinstance(count, int):  # 값이 정수인 경우에만 처리
            reaction_count += count
            
    reactions[date] = reaction_count    # 각 날짜별 총 반응 수를 딕셔너리에 저장
    
reactions

##### 딕셔너리를 데이터프레임으로 변환

In [ ]:
df_reaction = pd.DataFrame(list(reactions.items()), columns=["날짜", "반응도"])
df_reaction

##### 인덱스 재설정

In [ ]:
df_reaction.set_index("날짜", inplace=True)
df_reaction

### 2. 삼성전자 기사 카테고리별 합계

##### 삼성전자 기사 추출

In [ ]:
df_category = df_news[df_news["content"].str.contains("삼성전자")]
df_category

##### category열 이름 변경

In [ ]:
df_category.rename(columns={"category":"카테고리"}, inplace=True)
df_category

##### 카테고리별 합계

In [ ]:
count_category = df_category["카테고리"].value_counts()
count_category

##### 카테고리별 합계 이름 변경

In [ ]:
count_category.name = "본문"
count_category

## 시각화

#### 라이브러리

##### 라이브러리 설치

In [ ]:
!pip3 install matplotlib

##### 라이브러리 불러오기

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

# 한글 폰트 설정
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

### 1. 기사 버즈량 및 주가 비교 그래프
버즈량: 온라인상 실제 고객들을 통해 언급된 횟수

##### 그래프에 필요한 데이터 불러오기

In [ ]:
samsung

##### x축 데이터 설정

In [ ]:
x = samsung.index
x

##### 기사 버즈량 그래프 y축 데이터 설정

In [ ]:
article_y = samsung["본문"].values
article_y

##### 주가 그래프 y축 데이터 설정

In [ ]:
stock_y = samsung["주가"].values
stock_y

##### 기사 버즈량 및 주가 비교 그래프

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1)

# 그래프 제목
ax.set_title('기사 버즈량 및 주가 비교') 

# 기사 버즈량 그래프 설정
ax.plot(x, article_y, 'blue', label='버즈량')

# 주가 그래프 설정
scaled_stock_y = [stock / 10000 for stock in stock_y] # 주가를 10000으로 나누어 스케일 조정 
ax.plot(x, scaled_stock_y, 'orange', label='주가')

fig.autofmt_xdate() # x축 레이블 텍스트 회전
ax.legend() # 범례 설정

plt.show() # 그래프 출력

### 2. 일자별 기사 관심도 그래프

##### 그래프에 필요한 데이터 불러오기

In [ ]:
df_reaction

##### x축 데이터 설정

In [ ]:
x = df_reaction.index
x

##### y축 데이터 설정

In [ ]:
y = df_reaction["반응도"].values
y

##### 일자별 기사 관심도 그래프

In [ ]:
plt.plot(x, y, 'green')

plt.xticks(rotation=70)

plt.show()

### 3. 카테고리별 기사량 그래프

##### 그래프에 필요한 데이터 불러오기

In [ ]:
categories = df_category["카테고리"].value_counts()
categories

##### 카테고리 순서 재정렬

In [ ]:
ordered = ['IT', '경제', '사회', '생활', '세계', '스포츠', '오피니언', '정치']
ordered_categories = categories.reindex(ordered)
ordered_categories

##### x축 데이터 설정

In [ ]:
category_x = ordered_categories.index
category_x

In [ ]:
category_y = ordered_categories.values
category_y

##### 카테고리별 기사량 그래프

In [ ]:
plt.bar(category_x, category_y) # 막대 그래프

plt.title('카테고리별 기사량') # 그래프 제목 설정

plt.show()

## 워드 클라우드

#### 라이브러리

##### 라이브러리 설치

In [ ]:
%pip install wordcloud

##### 라이브러리 불러오기

In [ ]:
from wordcloud import WordCloud, STOPWORDS

### 1. 기사 버즈량이 가장 많은 날 워드 클라우드

##### 기사 버즈량이 가장 많은 날 파악

In [ ]:
desc_samsung = samsung.sort_values("본문", ascending=False)
desc_samsung

##### 기사 버즈량이 가장 많은 날 기사 본문 추출

In [ ]:
desc_news = df_news[df_news["content"].str.contains("삼성전자")] # 삼성전자 포함된 기사 본문 추출
desc_news = desc_news[desc_news["date"] == "20230809"] # 기사 버즈량이 가장 많은 날 기사 추출
words = desc_news["content"].values # 기사 버즈량이 가장 많은 날 기사 본문 추출

##### 워드 클라우드 생성

In [ ]:
wordcloud = WordCloud(max_font_size = 200, font_path = '/Library/Fonts/AppleGothic', background_color = '#FFFFFF', width = 1200, height = 800).generate(' '.join(words))

plt.figure(figsize=(5, 5)) # 워드 클라우드 사이즈 설정
plt.imshow(wordcloud)
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()

# 1차 데이터 분석 회고

> # Stella's opinion
- 기사 버즈량이 가장 많은 날의 기사 본문을 그대로 사용해서 워드클라우드를 생성해보니, 불필요한 단어들이 너무 많이 포함되어 있음을 파악.
- 핵심 키워드 추출을 위해서는 텍스트 마이닝이 선행되어야 한다고 판단.
    - 불필요한 데이터 제거
    - 형태소 분석

# 2차 데이터 분석 진행 - 텍스트 마이닝

## 텍스트 마이닝

### 불필요한 데이터(결측값, 중복) 제거

##### 원본 데이터 불러오기

In [ ]:
df = pd.read_csv('naver_news/naver_news_20230801_20230831.csv')
df.head()

##### 결측값 확인

In [ ]:
# 각 열마다 존재하는 결측값 개수 카운트
df.isnull().sum()

##### 결측값 제거

In [ ]:
df.dropna(inplace=True)

##### 중복 확인

In [ ]:
df['content'].nunique()

##### 중복 데이터 제거

In [ ]:
df.drop_duplicates(subset=['content'], inplace=True)

##### 삼성전자가 포함된 본문 추출

In [ ]:
df = df[df['content'].str.contains('삼성전자')]
df

##### 날짜 형식 변경

In [ ]:
df['date'] = df['date'].apply(lambda x: x.split()[0].replace('-', ''))

##### 인덱스 재설정

In [ ]:
df.set_index('date', inplace=True)
df

##### 최종 데이터 출력

In [ ]:
pd.set_option('display.max_rows', None) # row 생략 없이 출력
pd.set_option('display.max_columns', None) # col 생략 없이 출력
pd.set_option('display.max_colwidth', None)  # col 너비 무제한

df['content']

#### 형태소 분석기

##### 형태소 분석기 KoNLPy 설치

In [ ]:
%pip install konlpy

##### 형태소 분석기 불러오기

In [ ]:
from konlpy.tag import Okt

##### 형태소 분석

In [ ]:
okt = Okt() # 명사 분석기에 이름 부여

df['tokenized'] = df['content'].apply(okt.nouns)
df['tokenized'] = df['tokenized'].apply(lambda x: [item for item in x if len(item)>1]) # 한 글자 제거
df['tokenized']

#### 사용자 사전 생성 후 단어 추가

##### 사용자 사전 파일 경로

In [ ]:
user_dict_path = '/Users/fininsight/anaconda3/envs/stella/lib/python3.11/site-packages/konlpy/java/org/openkoreantext/processor/util/noun/samsung_electronics.txt'

##### 사용자 사전 확인

In [ ]:
# 파일을 읽기 모드로 열어서 내용을 출력
with open(user_dict_path, 'r') as f:
    data = f.read()

data

##### 사용자 사전에 단어 추가

In [ ]:
# data += '\n'

##### 사용자 사전 정렬

In [ ]:
# 개행 문자를 기준으로 문자열을 리스트로 반환
words = data.split('\n')


# 빈 문자열을 리스트에서 제거
words = [word for word in words if word]


# 가나다 순으로 정렬
sorted_words = sorted(words)


# 정렬된 결과 출력
sorted_words

##### 수정된 사용자 사전 다시 저장

In [ ]:
with open(user_dict_path, 'w') as f:
   f.write(data)

#### 사용자 사전을 적용한 형태소 분석

##### Okt 객체 초기화

In [ ]:
okt = Okt(user_dict_path)

##### 사용자 정의 명사 사전 파일을 Okt 객체에 추가

In [ ]:
with open('/Users/fininsight/anaconda3/envs/stella/lib/python3.11/site-packages/konlpy/java/org/openkoreantext/processor/util/noun/nouns.txt', 'r', encoding='utf-8') as f:
    okt_nouns = f.readlines()
    for line in okt_nouns:
        word = line.strip() # 개행 문자 제거

        with open(user_dict_path, 'a', encoding='utf-8') as user_file:
            user_file.write(word + '\n')